### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

### Read all the pdf's inside the directory

In [7]:

def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 1 PDF files to process

Processing: A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf
  ✓ Loaded 9 pages

Total documents loaded: 9


In [8]:
all_pdf_documents

[Document(metadata={'producer': 'Adobe PDF Library 23.8.53', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2024-02-21T12:54:50-06:00', 'author': 'JOFSR', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-02-21T12:55:38-06:00', 'sourcemodified': 'D:20240221185350', 'subject': '', 'title': '', 'source': '..\\data\\pdf\\A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'file_type': 'pdf'}, page_content='A Retrieval-Augmented Generation Based Large \nLanguage Model Benchmarked on a Novel Dataset \nKieran Pichai \nMenlo School \nAB\nSTRACT \nThe evolution of natural language processing has seen marked advancements, particularly with the advent of models \nlike BERT, Transformers, and GPT variants, with recent additions like GPT  and Bard. This paper investiga

### Text splitting get into chunks

In [9]:
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [10]:
chunks=split_documents(all_pdf_documents)
chunks

Split 9 documents into 41 chunks

Example chunk:
Content: A Retrieval-Augmented Generation Based Large 
Language Model Benchmarked on a Novel Dataset 
Kieran Pichai 
Menlo School 
AB
STRACT 
The evolution of natural language processing has seen marked advanc...
Metadata: {'producer': 'Adobe PDF Library 23.8.53', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2024-02-21T12:54:50-06:00', 'author': 'JOFSR', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-02-21T12:55:38-06:00', 'sourcemodified': 'D:20240221185350', 'subject': '', 'title': '', 'source': '..\\data\\pdf\\A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Adobe PDF Library 23.8.53', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2024-02-21T12:54:50-06:00', 'author': 'JOFSR', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-02-21T12:55:38-06:00', 'sourcemodified': 'D:20240221185350', 'subject': '', 'title': '', 'source': '..\\data\\pdf\\A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'file_type': 'pdf'}, page_content='A Retrieval-Augmented Generation Based Large \nLanguage Model Benchmarked on a Novel Dataset \nKieran Pichai \nMenlo School \nAB\nSTRACT \nThe evolution of natural language processing has seen marked advancements, particularly with the advent of models \nlike BERT, Transformers, and GPT variants, with recent additions like GPT  and Bard. This paper investiga

### embedding And vectorStoreDB

In [11]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

d:\CampusX\RAG-using-Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\CampusX\RAG-using-Langchain\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\subhadip\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer 

Model loaded successfully. Embedding dimension: 384


### VectorStore

In [13]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [14]:
chunks

[Document(metadata={'producer': 'Adobe PDF Library 23.8.53', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2024-02-21T12:54:50-06:00', 'author': 'JOFSR', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-02-21T12:55:38-06:00', 'sourcemodified': 'D:20240221185350', 'subject': '', 'title': '', 'source': '..\\data\\pdf\\A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'A retrieval-augmented generation based large language model benchmarked on a novel dataset.pdf', 'file_type': 'pdf'}, page_content='A Retrieval-Augmented Generation Based Large \nLanguage Model Benchmarked on a Novel Dataset \nKieran Pichai \nMenlo School \nAB\nSTRACT \nThe evolution of natural language processing has seen marked advancements, particularly with the advent of models \nlike BERT, Transformers, and GPT variants, with recent additions like GPT  and Bard. This paper investiga

In [15]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generate the Embeddings
embeddings = embedding_manager.generate_embeddings(texts)

### store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 41 texts...


Batches: 100%|██████████| 2/2 [00:17<00:00,  8.61s/it]


Generated embeddings with shape: (41, 384)
Adding 41 documents to vector store...
Successfully added 41 documents to vector store
Total documents in collection: 41


### Retriever Pipeline From VectorStore

In [16]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [17]:
rag_retriever

In [21]:
rag_retriever.retrieve("Advancements in RAG Framework", top_k=1, score_threshold=0.1)

Retrieving documents for query: 'Advancements in RAG Framework'
Top K: 1, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.41it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]